In [ ]:
import pandas as pd

from bs4  import BeautifulSoup
from time import sleep


from selenium                          import webdriver
from webdriver_manager.chrome          import ChromeDriverManager
from selenium.webdriver.support.select import Select
from selenium.common.exceptions        import NoSuchElementException

In [ ]:
pd.set_option("display.max_columns", 100)

# 시즌별 선수 성적 수집

In [ ]:
url = "https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx"
print(url)

In [ ]:
"""
시즌별 선수 성적 크롤링
"""

driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
driver.get(url)
sleep(1)

season_xpath       = '//*[@id="cphContents_cphContents_cphContents_ddlSeason_ddlSeason"]'
basic_info_xpath_1 = '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/div[1]/ul/li[1]/a'
basic_info_xpath_2 = '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/ul/li[1]/a'
detail_info_xpath  = '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/div[1]/ul/li[2]/a'
first_info_xpath   = '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/div[2]/a[1]'
second_info_xpath  = '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/div[2]/a[2]'
all_season_df = []
info_types    = ["basic_1", "basic_2", "detail"]

# seasons = list(range(2002, 2022))
seasons = list(range(2008, 2009))

# 02년 ~ 21년, 통산 성적
# 세부 기록에서는 '전체'를 고를 수가 없어서 기본사항으로 넘어간 후 전체를 고른다
for year in seasons:
    season = driver.find_element_by_xpath(season_xpath)
    Select(season).select_by_value(str(year))
    sleep(0.5)
    
    # 한 연도의 각 정보 카테고리의 테이블 리스트
    a_season_df = []
    
    for info_name in info_types:
        # 기본 정보일 경우
        if (info_name == "basic_1") or (info_name == "basic_2"):

            # 기본 정보 클릭
            try:                
                driver.find_element_by_xpath(basic_info_xpath_1).click()
                sleep(0.5)
            except NoSuchElementException as e:
                driver.find_element_by_xpath(basic_info_xpath_2).click()
                sleep(0.5)
            
            if info_name == "basic_1":
                driver.find_element_by_xpath(first_info_xpath).click()
                sleep(0.5)
            elif info_name == "basic_2":
                driver.find_element_by_xpath(second_info_xpath).click()
                sleep(0.5)
                
        # 상세 정보일 경우
        else:
            driver.find_element_by_xpath(detail_info_xpath).click()
            sleep(0.5)
        
        # 모든 페이지의 테이블 가져오기
        # len_page 임의로 3 설정
        page_tables = []
        page        = 1
        len_page    = 3
        while page < len_page - 1:
            pages    = driver.find_element_by_class_name('paging')
            pages    = pages.find_elements_by_tag_name('a')
            len_page = len(pages)

            pages[page].click()
            sleep(1)
        
            # table 가져오기
            html  = BeautifulSoup(driver.page_source, 'lxml')
            table = pd.read_html(str(html), encoding='cp949')[0]
            table["YEAR"] = year

            page_tables.append(table)
            page += 1
        
        # 한 연도에서 한 정보 카테고리의 모든 페이지들을 통합시킨다.
        table = pd.concat(page_tables, axis=0).reset_index(drop=True)
        a_season_df.append(table)
    
    # 한 연도에서 모든 정보 카테고리들을 통합시킨다.
    basic_df_1 = a_season_df[0]
    basic_df_2 = a_season_df[1]
    detail_df  = a_season_df[2]
    all_season_df.append(pd.merge(pd.merge(basic_df_1, basic_df_2), detail_df, on=["선수명", "순위", "팀명", "AVG", "YEAR"], how='left'))
    sleep(1)
    
driver.quit()

# kbo 통산성적 수집

In [ ]:
"""
kbo 통산성적
"""

all_season_df = each_all_season_df[:]

# 비율은 모두 float형으로 나타내기 때문에, float에겐 평균값을, '순위' column을 제외한 int형의 모든 열은 sum으로 합산해준다.
rate_cols = all_season_df[0].dtypes[all_season_df[0].dtypes == "float"].index.tolist() 
int_cols  = all_season_df[0].dtypes[all_season_df[0].dtypes == "int"].index.tolist()
fix_int_cols = ['순위', 'YEAR']
for col in fix_int_cols:
    int_cols.remove(col)

for idx in range(len(all_season_df) - 1):
    print(idx)
    cur_df  = all_season_df[idx]
    next_df = all_season_df[idx + 1]
    
    table = cur_df.merge(next_df, how='outer')
    duplicated_names = table["선수명"][table["선수명"].duplicated()]
    
    for name in duplicated_names:
        duplicated_pair = table["선수명"][table["선수명"] == name].index
        first_idx  = duplicated_pair[0]
        second_idx = duplicated_pair[1]
        
        # float 열 계산
        mean_of_pair = (table.loc[first_idx, rate_cols] + table.loc[second_idx, rate_cols]) / 2
        table.loc[first_idx, rate_cols] = mean_of_pair.apply(round, args=(3, ))
        
        # int열 계산
        sum_of_pair = table.loc[first_idx, int_cols] + table.loc[second_idx, int_cols]
        table.loc[first_idx, int_cols] = sum_of_pair
        
        table = table.drop(second_idx, axis=0)
        table = table.reset_index(drop=True)
        
    all_season_df[idx + 1] = table
    
concat_df = all_season_df[-1]

In [ ]:
# 시즌별 선수가 kbo 전체 선수를 가지고 있지 않다. ex) 08 이명기가 통산성적에는 존재하나, 시즌 성적의 명단에는 존재하지 않음.
# temp_all_season_df[6][temp_all_season_df[6]["선수명"] == "이명기"]

# 시즌별 성적 및 통산 성적 csv화

In [ ]:
# 시즌별 성적 csv화 시키기
for idx, df in enumerate(each_all_season_df):
    year = seasons[idx]
    df.to_csv(f"data/kbo_{year}_season.csv", index=False)

In [ ]:
concat_df.to_csv('data/all_kbo.csv', index=False)

In [ ]:
pd.read_csv('data/kbo_2017_season.csv').head()